# Data preparation with Databao – Web shop orders demo (Case 1)

Welcome! This notebook will walk you through a practical end‑to‑end data preparation workflow for a webshop orders dataset using [Databao](https://databao.app).
You’ll learn how to transform transactional into clean, structured, and insight-ready outputs.

The notebook contains the typical data-preparation steps:
> Understanding → Cleaning → Integration → Feature engineering → Aggregation & export → Insights

This corresponds to the following progression:
> “What data do we have?” → “Is it clean?” → “Can we aggregate and group it?” → “What KPIs can we compute?” → “What are the trends and drivers?” → “What actions do we take?”

The notebook contains a DuckDB file with a sample dataset, and it can be used with both local and cloud LLMs. You can learn more about connecting to data, using LLMs, and running Databao in the [Databao docs](https://jetbrains.github.io/databao-docs/).

Let’s dive in!

In [ ]:
# Install Databao and other packages (safe to re-run)
!pip install -q duckdb databao matplotlib pandas


In [1]:
# Import packages
import os
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

# Connect to the local DuckDB file. `read_only=False` allows registering temp views/DFs.
DB_PATH = "data/web_shop_raw.duckdb"
if not os.path.exists(DB_PATH):
    raise FileNotFoundError(
        f"Expected DuckDB at {DB_PATH}. If you don't have it, check the README for setup instructions."
    )
conn = duckdb.connect(DB_PATH, read_only=False)
print(f"Connected to DuckDB database: {DB_PATH}")


Connected to DuckDB database: data/web_shop_raw.duckdb


In [2]:
# Import Databao and LLM config
import databao
from databao import LLMConfig

### 1. Configure your LLM

Databao supports both cloud and local LLMs.
For this demo, it’s easier and faster to use an OpenAI cloud model, but it requires an API key.

If you prefer to use a local model, all your data remains on your machine, but downloading a model may take some time. Depending on the model you use and your machine specs, generating answers may be slower compared to a cloud model.

For easier setup, this notebook uses a cloud LLM by default. If you prefer to use a local LLM, uncomment the corresponding section and comment out the line with the cloud LLM config.


In [ ]:
# Add your OpenAI API key. Comment out the following line if you prefer to use a local model
%env OPENAI_API_KEY=<YOUR_API_KEY>

In [6]:
# Option A — Cloud model (OpenAI). Low temperature helps produce deterministic SQL/plots
llm_config = LLMConfig(name="gpt-5.1", temperature=0)

# Option B — Local model (Ollama)
# llm_config = LLMConfig.from_yaml("../configs/qwen3-8b-ollama.yaml")  # Use a custom config file

### 2. Create a Databao agent and register sources and context

The following cell registers:
- A DuckDB connection
- A small per‑source schema overview (Markdown file)
- General project‑wide context via `agent.add_context()`


In [7]:
agent = databao.new_agent(llm_config=llm_config)

# Add a DuckDB connection with source context
agent.add_db(conn)

# Add additional project‑wide context (not tied to a specific source)
agent.add_context(
    """
    Project‑wide notes:
    - Monetary values are in EUR, unless stated otherwise.
    - Orders with a status of 'canceled' should be excluded from KPIs unless they're explicitly requested.
    - Treat obviously fake/test rows as data quality issues (e.g., emails like test@, or id like 'Test User').
    - Date columns use UTC timestamps unless noted; compute delivery_days from purchase to delivered_customer.
    """
)

print("Registered DBs:", list(agent.dbs.keys()))


Registered DBs: ['db1']


### 3) Start a thread

- `.ask(prompt)` executes immediately (eager mode) and materializes results.
- If you want to chain several `ask()` calls before executing them, you can switch to lazy mode using `session.thread(lazy=True)`.


In [8]:
thread = agent.thread()

## Step 1 — Understanding data

Question: What data do we have, and how are the tables connected?
Action: Explore schema (orders, items, products, payments, reviews, customers, sellers).
Outcome: Identify key joins (order_id, customer_id, seller_id).


In [9]:
thread.ask(
    """
    Provide a concise schema overview of our webshop database. Include the following:
    - List of core tables (orders, order_items, products, payments, reviews, customers, sellers).
    - Primary keys and foreign keys per table.
    - A short note on how to join them at the order level.
    Return a short Markdown/text summary, keep it tight.
    """
)


======== <THINKING> ========

Here’s a concise schema overview for the webshop database.

---

### Core tables and keys

**orders**  
- PK: `order_id`  
- FKs:  
  - `customer_id` → `customers.customer_id`  

**order_items**  
- PK: composite (`order_id`, `order_item_id`)  
- FKs:  
  - `order_id` → `orders.order_id`  
  - `product_id` → `products.product_id`  
  - `seller_id` → `sellers.seller_id`  

**products**  
- PK: `product_id`  
- FKs:  
  - `product_category_name` → `product_category_name_translation.product_category_name` (lookup for English name)  

**order_payments**  
- PK: composite (`order_id`, `payment_sequential`)  
- FKs:  
  - `order_id` → `orders.order_id`  

**order_reviews**  
- PK: `review_id`  
- FKs:  
  - `order_id` → `orders.order_id`  

**customers**  
- PK: `customer_id`  
- Other important columns:  
  - `customer_zip_code_prefix` → `geolocation.geolocation_zip_code_prefix` (location info)  

**sellers**  
- PK: `seller_id`  
- Other important columns:  
 

Materialized Thread with 0 data rows.

In [10]:
display(Markdown(thread.text()))

Here’s a concise schema overview for the webshop database.

---

### Core tables and keys

**orders**  
- PK: `order_id`  
- FKs:  
  - `customer_id` → `customers.customer_id`  

**order_items**  
- PK: composite (`order_id`, `order_item_id`)  
- FKs:  
  - `order_id` → `orders.order_id`  
  - `product_id` → `products.product_id`  
  - `seller_id` → `sellers.seller_id`  

**products**  
- PK: `product_id`  
- FKs:  
  - `product_category_name` → `product_category_name_translation.product_category_name` (lookup for English name)  

**order_payments**  
- PK: composite (`order_id`, `payment_sequential`)  
- FKs:  
  - `order_id` → `orders.order_id`  

**order_reviews**  
- PK: `review_id`  
- FKs:  
  - `order_id` → `orders.order_id`  

**customers**  
- PK: `customer_id`  
- Other important columns:  
  - `customer_zip_code_prefix` → `geolocation.geolocation_zip_code_prefix` (location info)  

**sellers**  
- PK: `seller_id`  
- Other important columns:  
  - `seller_zip_code_prefix` → `geolocation.geolocation_zip_code_prefix`  

---

### How to join at the order level

Typical order‑level join pattern:

```sql
FROM orders o
LEFT JOIN customers c
       ON o.customer_id = c.customer_id
LEFT JOIN order_items oi
       ON o.order_id = oi.order_id
LEFT JOIN products p
       ON oi.product_id = p.product_id
LEFT JOIN sellers s
       ON oi.seller_id = s.seller_id
LEFT JOIN order_payments op
       ON o.order_id = op.order_id
LEFT JOIN order_reviews r
       ON o.order_id = r.order_id
```

For strictly order‑level KPIs (1 row per order), aggregate `order_items` (e.g., sum price, freight, item count) and `order_payments` (e.g., sum payment_value) before or after joining, and treat `order_reviews` as 0–1 row per order (e.g., `MAX(review_score)`).

In [11]:
# Ask for quick row counts to gauge table sizes
thread.ask(
    """
    Return a dataframe of row counts for the main tables
    """
)

======== <THINKING> ========

I will compute row counts for each main table in a single query using `UNION ALL` to return a compact summary as one dataframe.

[tool_call: 'run_sql_query']
```
{"sql":"SELECT 'orders' AS table_name, COUNT(*) AS row_count FROM db1.main.orders\nUNION ALL\nSELECT 'order_items' AS table_name, COUNT(*) AS row_count FROM db1.main.order_items\nUNION ALL\nSELECT 'products' AS table_name, COUNT(*) AS row_count FROM db1.main.products\nUNION ALL\nSELECT 'order_payments' AS table_name, COUNT(*) AS row_count FROM db1.main.order_payments\nUNION ALL\nSELECT 'order_reviews' AS table_name, COUNT(*) AS row_count FROM db1.main.order_reviews\nUNION ALL\nSELECT 'customers' AS table_name, COUNT(*) AS row_count FROM db1.main.customers\nUNION ALL\nSELECT 'sellers' AS table_name, COUNT(*) AS row_count FROM db1.main.sellers\nORDER BY table_name;"}
```


```sql
SELECT 'orders' AS table_name, COUNT(*) AS row_count FROM db1.main.orders
UNION ALL
SELECT 'order_items' AS table_name, C

,table_name,row_count
0,customers,203
1,order_items,1352
2,order_payments,1409
3,order_reviews,172
4,orders,606
5,products,106
6,sellers,55


In [12]:
row_counts = thread.df()
row_counts

,table_name,row_count
0,customers,203
1,order_items,1352
2,order_payments,1409
3,order_reviews,172
4,orders,606
5,products,106
6,sellers,55


## Step 2 — Cleaning and validation

Question: Are there duplicates or missing values?
Action: Fix data types, clean nulls, remove test rows.
Outcome: Normalized, ready-to-merge datasets.

We will ask Databao to propose cleaning steps and produce cleaned, queryable outputs (views or temp tables) we can reuse.


In [28]:
thread.ask(
    """
    Check tables for common issues and propose pragmatic cleaning steps:
    - Duplicates by natural keys (e.g., one row per order_id in orders etc.).
    - Obvious type issues (timestamps, numerics).
    - Clearly fake/test rows (e.g., customers with emails like '%test%' or ids 'test%').
    show me tables with issues and write down what's wrong with them
    -
    """
)

======== <THINKING> ========

**Evaluating schema values**

Since the schema is already typed, I need to look for nonsensical values, like timestamps that are out of range, nulls in unexpected spots, or negative values for price and payment. I'll verify that review scores are within 1-5, and lengths, weights, and dimensions aren't negative. Also, I want to ensure that the delivery dates are in the correct order. Using functions.run_sql_query, I’ll look for duplicates and type anomalies efficiently by running multiple queries in parallel.**Planning duplicate and type checks**

I'm laying out my approach to identify duplicates in orders. First, I'll run a query to count duplicate rows and distinct order IDs. For example, to get the exact duplicates, I’ll use a subquery to check how many times each order ID appears. I also want to check for chronology issues and nulls in key fields, ensuring the order dates make sense and fall within valid ranges. I'll also note cases where delivery dates

,test_id_rows,test_title_rows,test_message_rows
0,0.0,0.0,0.0


In [29]:
display(Markdown(thread.text()))

Here are **only the tables with issues**, plus pragmatic cleaning steps.

---

## 1) `orders`

**What’s wrong**

- **Duplicate natural key**
  - Expected: 1 row per `order_id`.
  - Found: multiple `order_id` values with more than one row (3 extra rows in total).
- **Date consistency**
  - No impossible sequences (approval/delivery before purchase).
  - But many orders where  
    `order_delivered_customer_date > order_estimated_delivery_date` → **late vs ETA**, not a type error.

**Pragmatic cleaning**

- Build a **deduped orders table/view**:
  - 1 row per `order_id`, using `ROW_NUMBER() OVER (PARTITION BY order_id ORDER BY <best timestamp desc>)` and keep `rn = 1`.
  - Optionally exclude `order_status = 'canceled'` for KPI reporting.
- Add delivery metrics instead of dropping late orders:
  - `delivery_days` = days from purchase to delivery.
  - `is_late` flag where delivered > estimated.
  - `delivery_delay_days` = delivered − estimated.

Use this as `orders_clean` in all downstream joins.

---

## 2) `products`

**What’s wrong**

- **Duplicate natural key**
  - Expected: 1 row per `product_id`.
  - Found: some `product_id` values with multiple rows (6 extra rows total).

**Pragmatic cleaning**

- Create **`products_clean`**:
  - 1 row per `product_id` via `ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY <completeness desc>)`.
  - Prefer rows with more non‑null attributes (weight, dimensions, description length, photos).
- Enforce uniqueness of `product_id` in this cleaned layer, keep raw table as landing.

---

## 3) `sellers`

**What’s wrong**

- **Duplicate natural key**
  - Expected: 1 row per `seller_id`.
  - Found: some `seller_id` values with multiple rows (5 extra rows total).

**Pragmatic cleaning**

- Create **`sellers_clean`**:
  - 1 row per `seller_id` using `ROW_NUMBER() OVER (PARTITION BY seller_id ORDER BY <location completeness desc>)`.
  - Prefer rows with non‑null `seller_zip_code_prefix`, `seller_city`, `seller_state`.

---

## 4) `customers`

**What’s wrong**

- **Test / fake IDs**
  - Found **3 customers** where `customer_id` contains `'test'` (e.g. `LOWER(customer_id) LIKE '%test%'`).
  - No obvious “test” cities.

**Pragmatic cleaning**

- Build **`customers_clean`**:
  - Exclude test customers: `WHERE LOWER(customer_id) NOT LIKE '%test%'`.
  - Optionally keep them in a separate `customers_test` table for QA.
- In analytics, always join against `customers_clean` instead of the raw table.

---

## What looks OK (for context)

- No duplicates for:
  - `order_items` on (`order_id`, `order_item_id`)
  - `order_payments` on (`order_id`, `payment_sequential`)
  - `order_reviews` on `review_id`
  - `customers` on `customer_id` (besides test IDs)
- Numeric ranges:
  - `price`, `freight_value`, `payment_value` are positive.
  - No negative/zero installments.
  - Product weights and dimensions are > 0.
  - `review_score` always in **[1, 5]**.
- Timestamps:
  - Within a reasonable range; no purchase/approval/delivery order inversions, only late vs ETA.

In practice, use `orders_clean`, `products_clean`, `sellers_clean`, and `customers_clean` as your **modeled layer**, and keep the raw tables as a staging/landing layer.

In [15]:
thread.ask(
    """
    give me sql to fix issues, mentioned above
    """
)

======== <THINKING> ========

**Developing SQL solutions**

The user wants SQL queries to address several issues like duplicates in orders, products, and sellers, as well as modeling around late deliveries. They are asking for solutions, likely involving DDL/CTAS or view creation to clean these dimensions. I need to keep it concise and clear. For instance, I’ll create a cleaned orders table that eliminates duplicates based on a deterministic rule, using row numbers with relevant timestamps. Additionally, there should be a cleaned products table that chooses the most complete rows based on specific characteristics.**Clarifying SQL requirements**

To simplify the deduplication of products, I should order by product description length, number of photos, weight, and then product ID for determinism. For cleaning sellers, I'll also deduplicate by seller ID, but I need to ensure stability in my ordering. I'll use row numbers to pick the first entry based on multiple columns to maintain determ

Materialized Thread with 0 data rows.

## Step 3 — Integration

Question: Can we combine customers, sellers, payments, and reviews?
Action: Join entities into a consolidated dataset.
Outcome: Unified order-level table with spend, freight, category, seller location, and reviews.


In [16]:
thread.ask(
    """
    Build a unified order-level dataset from the cleaned tables with one row per order_id. Include at least:
    - order_id, order_purchase_timestamp, order_approved_at
    - customer_id, customer_city, customer_state
    - seller_id (dominant seller for the order if multiple), seller_city, seller_state
    - items_count, total_items_price, total_freight_value, total_payment_value
    - main_product_category_name (mode across items), review_score
    Name this output orders_unified (as a DuckDB temp table) and also return it as a DataFrame sample (head).
    """
)


======== <THINKING> ========

**Exploring DuckDB temp tables**

I’m trying to clarify the syntax for creating a temporary table in DuckDB: "CREATE TEMP TABLE orders_unified AS SELECT ...". The default schema is `temp.main`, and the tool environment refers to DataFrames being located there. For temp tables, it uses the `temp` schema without a specific name, like `temp.main.orders_unified`. I'll need to compute one row per `order_id`, using deduped `orders_clean`, and implement the ranking logic in a CTE. I'll also join to customers and determine the dominant `seller_id` based on item count or total price when there are multiple sellers.**Calculating aggregated metrics**

I need to compute aggregated metrics per order from the `order_items`. This includes counting distinct `order_item_id` for `items_count`, summing prices for `total_items_price`, and summing freight values for `total_freight_value`. I also want to find the `main_product_category_name` by determining the mode across items

,order_id,order_purchase_timestamp,order_approved_at,customer_id,customer_city,customer_state,seller_id,seller_city,seller_state,items_count,total_items_price,total_freight_value,total_payment_value,main_product_category_name,review_score
0,ord_0253,2025-06-02 02:03:00,2025-06-02 04:23:00,cust_0053,arnhem,GE,sell_0024,utrecht,UT,1,196.38,39.92,201.93,Pets,<NA>
1,ord_0059,2025-06-02 04:02:00,2025-06-02 05:48:00,cust_0059,roermond,LI,sell_0007,vlissingen,ZL,4,1235.63,156.46,1392.09,"Bed, Bath & Table",<NA>
2,ord_0119,2025-06-02 04:31:00,2025-06-02 04:48:00,cust_0119,groningen,GR,sell_0018,rotterdam,ZH,4,2876.70,171.68,3048.38,Furniture & Decor,<NA>
3,ord_0098,2025-06-02 05:08:00,2025-06-02 05:37:00,cust_0098,rotterdam,ZH,sell_0035,zaandam,NH,3,878.88,99.33,915.79,Watches & Gifts,<NA>
4,ord_0306,2025-06-02 09:06:00,2025-06-02 10:04:00,cust_0106,kampen,OV,sell_0045,drachten,FR,1,120.30,56.80,140.26,Sports & Leisure,2
5,ord_0305,2025-06-02 09:12:00,2025-06-02 11:41:00,cust_0105,drachten,FR,sell_0006,zwolle,OV,4,1579.57,185.90,1633.11,Computers,2
6,ord_0427,2025-06-02 11:26:00,2025-06-02 12:33:00,cust_0027,leeuwarden,FR,sell_0002,maastricht,LI,1,143.19,40.28,183.47,Office Supplies,4
7,ord_0254,2025-06-02 15:55:00,2025-06-02 16:15:00,cust_0054,drachten,FR,sell_0016,dronten,FL,8,3647.69,278.62,3780.19,Beauty & Health,<NA>
8,ord_0384,2025-06-02 16:00:00,2025-06-02 18:37:00,cust_0184,rotterdam,ZH,sell_0029,heerlen,LI,5,1171.57,167.30,1226.18,Car,<NA>
9,ord_0507,2025-06-03 05:56:00,2025-06-03 06:39:00,cust_0107,zoetermeer,ZH,sell_0045,drachten,FR,1,85.51,20.52,90.67,Fashion & Shoes,<NA>


In [17]:
orders_unified = thread.df()
orders_unified

,order_id,order_purchase_timestamp,order_approved_at,customer_id,customer_city,customer_state,seller_id,seller_city,seller_state,items_count,total_items_price,total_freight_value,total_payment_value,main_product_category_name,review_score
0,ord_0253,2025-06-02 02:03:00,2025-06-02 04:23:00,cust_0053,arnhem,GE,sell_0024,utrecht,UT,1,196.38,39.92,201.93,Pets,<NA>
1,ord_0059,2025-06-02 04:02:00,2025-06-02 05:48:00,cust_0059,roermond,LI,sell_0007,vlissingen,ZL,4,1235.63,156.46,1392.09,"Bed, Bath & Table",<NA>
2,ord_0119,2025-06-02 04:31:00,2025-06-02 04:48:00,cust_0119,groningen,GR,sell_0018,rotterdam,ZH,4,2876.70,171.68,3048.38,Furniture & Decor,<NA>
3,ord_0098,2025-06-02 05:08:00,2025-06-02 05:37:00,cust_0098,rotterdam,ZH,sell_0035,zaandam,NH,3,878.88,99.33,915.79,Watches & Gifts,<NA>
4,ord_0306,2025-06-02 09:06:00,2025-06-02 10:04:00,cust_0106,kampen,OV,sell_0045,drachten,FR,1,120.30,56.80,140.26,Sports & Leisure,2
5,ord_0305,2025-06-02 09:12:00,2025-06-02 11:41:00,cust_0105,drachten,FR,sell_0006,zwolle,OV,4,1579.57,185.90,1633.11,Computers,2
6,ord_0427,2025-06-02 11:26:00,2025-06-02 12:33:00,cust_0027,leeuwarden,FR,sell_0002,maastricht,LI,1,143.19,40.28,183.47,Office Supplies,4
7,ord_0254,2025-06-02 15:55:00,2025-06-02 16:15:00,cust_0054,drachten,FR,sell_0016,dronten,FL,8,3647.69,278.62,3780.19,Beauty & Health,<NA>
8,ord_0384,2025-06-02 16:00:00,2025-06-02 18:37:00,cust_0184,rotterdam,ZH,sell_0029,heerlen,LI,5,1171.57,167.30,1226.18,Car,<NA>
9,ord_0507,2025-06-03 05:56:00,2025-06-03 06:39:00,cust_0107,zoetermeer,ZH,sell_0045,drachten,FR,1,85.51,20.52,90.67,Fashion & Shoes,<NA>


## Step 4 — Feature engineering

Question: Which KPIs help analyze performance?
Action: Compute total_price, freight, delivery_days, delay_days, review_score.
Outcome: Metrics such as average delivery time, review score, and order value.


In [18]:
thread.ask(
    """
    From orders_unified, compute a feature-rich table orders_features with per-order KPI: come up with most important metrics, expain why you chose it
    """
)


======== <THINKING> ========

**Computing KPIs for orders**

The user wants me to create a feature-rich table called orders_features based on the orders_unified temp table. I’ll need to come up with the most important metrics and explain my choices. I’ll likely use functions.run_sql_query to create this table.

Considering the columns in orders_unified, we can derive metrics like delivery days and lateness, even though delivery fields are missing in the original table. Joining the orders table could help, but I'll aim to keep things simple by primarily using orders_unified plus original orders for useful KPIs like payment-to-item ratio and freight share.**Defining orders_features**

The temp orders_unified doesn't have delivery timestamps since those were removed, but the base orders table does. I could define orders_features using a SELECT statement with joins to db1.main.orders. It seems like creating a CTE called orders_clean could help in computing key metrics.

For metrics like gr

,order_id,order_purchase_timestamp,order_purchase_date,order_approved_at,customer_id,customer_city,customer_state,seller_id,seller_city,seller_state,items_count,total_items_price,total_freight_value,total_payment_value,main_product_category_name,review_score,order_delivered_customer_date,order_estimated_delivery_date,delivery_days,is_late,delivery_delay_days,order_gmv,avg_item_price,freight_share_pct,payment_gap,discount_pct,has_review,is_positive_review,is_negative_review,is_high_value_order
0,ord_0253,2025-06-02 02:03:00,2025-06-02,2025-06-02 04:23:00,cust_0053,arnhem,GE,sell_0024,utrecht,UT,1,196.38,39.92,201.93,Pets,<NA>,2025-06-15 17:03:00,2025-06-14 02:03:00,13,1,1,236.30,196.38000,16.893779,-34.37,14.545070,0,0,0,0
1,ord_0059,2025-06-02 04:02:00,2025-06-02,2025-06-02 05:48:00,cust_0059,roermond,LI,sell_0007,vlissingen,ZL,4,1235.63,156.46,1392.09,"Bed, Bath & Table",<NA>,2025-06-12 22:02:00,2025-06-18 04:02:00,10,0,-6,1392.09,308.90750,11.239216,0.00,NaN,0,0,0,1
2,ord_0119,2025-06-02 04:31:00,2025-06-02,2025-06-02 04:48:00,cust_0119,groningen,GR,sell_0018,rotterdam,ZH,4,2876.70,171.68,3048.38,Furniture & Decor,<NA>,NaT,2025-06-21 04:31:00,<NA>,0,<NA>,3048.38,719.17500,5.631844,0.00,NaN,0,0,0,1
3,ord_0098,2025-06-02 05:08:00,2025-06-02,2025-06-02 05:37:00,cust_0098,rotterdam,ZH,sell_0035,zaandam,NH,3,878.88,99.33,915.79,Watches & Gifts,<NA>,2025-06-15 15:08:00,2025-06-07 05:08:00,13,1,8,978.21,292.96000,10.154261,-62.42,6.381043,0,0,0,1
4,ord_0306,2025-06-02 09:06:00,2025-06-02,2025-06-02 10:04:00,cust_0106,kampen,OV,sell_0045,drachten,FR,1,120.30,56.80,140.26,Sports & Leisure,2,2025-06-14 05:06:00,2025-06-10 09:06:00,12,1,4,177.10,120.30000,32.072276,-36.84,20.801807,1,0,1,0
5,ord_0305,2025-06-02 09:12:00,2025-06-02,2025-06-02 11:41:00,cust_0105,drachten,FR,sell_0006,zwolle,OV,4,1579.57,185.90,1633.11,Computers,2,2025-06-14 06:12:00,2025-06-07 09:12:00,12,1,7,1765.47,394.89250,10.529774,-132.36,7.497154,1,0,1,1
6,ord_0427,2025-06-02 11:26:00,2025-06-02,2025-06-02 12:33:00,cust_0027,leeuwarden,FR,sell_0002,maastricht,LI,1,143.19,40.28,183.47,Office Supplies,4,2025-06-12 23:26:00,2025-06-12 11:26:00,10,1,0,183.47,143.19000,21.954543,0.00,NaN,1,1,0,0
7,ord_0254,2025-06-02 15:55:00,2025-06-02,2025-06-02 16:15:00,cust_0054,drachten,FR,sell_0016,dronten,FL,8,3647.69,278.62,3780.19,Beauty & Health,<NA>,2025-06-13 18:55:00,2025-06-18 15:55:00,11,0,-5,3926.31,455.96125,7.096230,-146.12,3.721560,0,0,0,1
8,ord_0384,2025-06-02 16:00:00,2025-06-02,2025-06-02 18:37:00,cust_0184,rotterdam,ZH,sell_0029,heerlen,LI,5,1171.57,167.30,1226.18,Car,<NA>,2025-06-13 21:00:00,2025-06-14 16:00:00,11,0,-1,1338.87,234.31400,12.495612,-112.69,8.416799,0,0,0,1
9,ord_0507,2025-06-03 05:56:00,2025-06-03,2025-06-03 06:39:00,cust_0107,zoetermeer,ZH,sell_0045,drachten,FR,1,85.51,20.52,90.67,Fashion & Shoes,<NA>,NaT,2025-06-07 05:56:00,<NA>,0,<NA>,106.03,85.51000,19.353013,-15.36,14.486466,0,0,0,0


In [19]:
display(Markdown(thread.text()))

Here’s what I’ve built in `orders_features` (DuckDB temp table) and *why* these KPIs are there. The table is already created; you can inspect it with:

```sql
SELECT * FROM orders_features LIMIT 10;
```

### Core grouping

- **Grain:** 1 row per `order_id` (taken from `orders_unified`, excluding canceled via `orders_clean` join).
- **Keys / context (for slicing):**
  - `order_id`, `order_purchase_timestamp`, `order_purchase_date`
  - `customer_id`, `customer_city`, `customer_state`
  - `seller_id`, `seller_city`, `seller_state`
  - `main_product_category_name`, `review_score`

These let you slice all KPIs by time, geography, seller, and category.

---

### Commercial KPIs

1. **Order GMV and components**
   - `items_count`  
   - `total_items_price`  
   - `total_freight_value`  
   - `total_payment_value`  
   - `order_gmv = total_items_price + total_freight_value`
   - **Why:** This is your basic revenue lens per order: how many items, pure merchandise vs shipping, and what the customer actually paid.

2. **Basket composition & pricing**
   - `avg_item_price = total_items_price / items_count`  
   - `freight_share_pct = 100 * total_freight_value / (total_items_price + total_freight_value)`  
   - **Why:**  
     - `avg_item_price` indicates whether it’s a low- or high-ticket basket.  
     - `freight_share_pct` shows how “expensive” delivery feels vs goods, a key driver of conversion and perceived fairness.

3. **Price vs payment (discounts / vouchers / extra charges)**
   - `payment_gap = total_payment_value - (total_items_price + total_freight_value)`  
   - `discount_pct` (only if `total_payment_value` < (items + freight)):  
     `100 * ((items + freight) - total_payment_value) / (items + freight)`  
   - **Why:**  
     - Captures coupons, discounts, rounding or surcharges.  
     - `discount_pct` is a clean, comparable metric across orders to analyze promo efficiency and margin pressure.

4. **High-value order indicator**
   - `is_high_value_order = 1` if `total_payment_value >= 500` (else 0)  
   - **Why:** Lets you quickly focus on orders that are most impactful financially (fraud checks, VIP flows, support priority, etc.). Threshold is business-tunable.

---

### Operational / logistics KPIs

5. **Delivery performance**
   - `order_delivered_customer_date`  
   - `order_estimated_delivery_date`  
   - `delivery_days = days from purchase to delivery`  
   - `is_late = 1 if delivered > estimated, else 0`  
   - `delivery_delay_days = delivered - estimated (positive = late, negative = early)`  
   - **Why:** These are the essential building blocks for:
     - SLA compliance (% on-time)
     - Average delivery time
     - Impact of late deliveries on reviews and repeat behavior.

---

### Customer experience KPIs

6. **Review / satisfaction flags**
   - `has_review = 1 if review_score not null`  
   - `is_positive_review = 1 if review_score >= 4`  
   - `is_negative_review = 1 if review_score <= 2`  
   - **Why:** Quick segmentation into:
     - Orders with feedback coverage (`has_review`)
     - Promoters vs detractors (high‑ vs low‑score buckets)
   - These make it easy to relate service metrics (e.g., `is_late`, `delivery_days`, `freight_share_pct`) to customer satisfaction.

---

### Recap: what `orders_features` gives you

Per non‑canceled order, you now have:

- Commercial: **GMV, discounting, basket size & structure**
- Operational: **delivery speed and SLA adherence**
- Experience: **review presence and sentiment flags**
- Context: **time, location, seller, category**

That’s a compact but expressive feature set for reporting, cohorting, or feeding into ML models (churn, CLV, NPS prediction, etc.).

In [20]:
orders_features = thread.df()
orders_features

,order_id,order_purchase_timestamp,order_purchase_date,order_approved_at,customer_id,customer_city,customer_state,seller_id,seller_city,seller_state,...,delivery_delay_days,order_gmv,avg_item_price,freight_share_pct,payment_gap,discount_pct,has_review,is_positive_review,is_negative_review,is_high_value_order
0,ord_0253,2025-06-02 02:03:00,2025-06-02,2025-06-02 04:23:00,cust_0053,arnhem,GE,sell_0024,utrecht,UT,...,1,236.30,196.38000,16.893779,-34.37,14.545070,0,0,0,0
1,ord_0059,2025-06-02 04:02:00,2025-06-02,2025-06-02 05:48:00,cust_0059,roermond,LI,sell_0007,vlissingen,ZL,...,-6,1392.09,308.90750,11.239216,0.00,NaN,0,0,0,1
2,ord_0119,2025-06-02 04:31:00,2025-06-02,2025-06-02 04:48:00,cust_0119,groningen,GR,sell_0018,rotterdam,ZH,...,<NA>,3048.38,719.17500,5.631844,0.00,NaN,0,0,0,1
3,ord_0098,2025-06-02 05:08:00,2025-06-02,2025-06-02 05:37:00,cust_0098,rotterdam,ZH,sell_0035,zaandam,NH,...,8,978.21,292.96000,10.154261,-62.42,6.381043,0,0,0,1
4,ord_0306,2025-06-02 09:06:00,2025-06-02,2025-06-02 10:04:00,cust_0106,kampen,OV,sell_0045,drachten,FR,...,4,177.10,120.30000,32.072276,-36.84,20.801807,1,0,1,0
5,ord_0305,2025-06-02 09:12:00,2025-06-02,2025-06-02 11:41:00,cust_0105,drachten,FR,sell_0006,zwolle,OV,...,7,1765.47,394.89250,10.529774,-132.36,7.497154,1,0,1,1
6,ord_0427,2025-06-02 11:26:00,2025-06-02,2025-06-02 12:33:00,cust_0027,leeuwarden,FR,sell_0002,maastricht,LI,...,0,183.47,143.19000,21.954543,0.00,NaN,1,1,0,0
7,ord_0254,2025-06-02 15:55:00,2025-06-02,2025-06-02 16:15:00,cust_0054,drachten,FR,sell_0016,dronten,FL,...,-5,3926.31,455.96125,7.096230,-146.12,3.721560,0,0,0,1
8,ord_0384,2025-06-02 16:00:00,2025-06-02,2025-06-02 18:37:00,cust_0184,rotterdam,ZH,sell_0029,heerlen,LI,...,-1,1338.87,234.31400,12.495612,-112.69,8.416799,0,0,0,1
9,ord_0507,2025-06-03 05:56:00,2025-06-03,2025-06-03 06:39:00,cust_0107,zoetermeer,ZH,sell_0045,drachten,FR,...,<NA>,106.03,85.51000,19.353013,-15.36,14.486466,0,0,0,0


## Step 5 — Aggregation, grouping, and export

Question: Can we summarize results per category or seller?
Action: Aggregate by category, month, and seller.
Outcome: Analytical dataset ready for KPI dashboards or modeling.


In [21]:
# Aggregate by month and product category
thread.ask(
    """
    From orders_features join back the main_product_category_name (if not already present) and aggregate by:
    month = date_trunc('month', order_date), product_category_name.
    Compute: orders_count, revenue_total, aov
    Return as df_category_month.
    """
)


======== <THINKING> ========

**Aggregating order data**

I need to work on aggregating data from orders_features, where they already have the main_product_category_name. I'll set the month by truncating the order_date to the month. I'll use order_purchase_date defined as DATE and identify the revenue_total likely as total_payment_value. The user wants revenue_total, excluding canceled orders. To simplify things, I'll use total_payment_value as revenue.  I’ll calculate orders_count, revenue_total, and average order value (AOV). I need to be mindful of the DuckDB syntax for date_trunc and ensure payments exist for each valid order.**Creating temp table for aggregation**

I'm thinking about how if there are nulls, the SUM will ignore them, but COUNT will still count all orders, which is good. This aligns with typical behavior. Now, I need to create a temporary table with a SQL command. I'll truncate the order_purchase_date to get the month, and I'll calculate the orders_count and revenue

,month,product_category_name,orders_count,revenue_total,aov
0,2025-06-01,Beauty & Health,8,6088.22,761.027500
1,2025-06-01,"Bed, Bath & Table",8,9505.27,1188.158750
2,2025-06-01,Car,8,11747.85,1468.481250
3,2025-06-01,Computers,12,24325.25,2027.104167
4,2025-06-01,Cool Stuff,19,36402.21,1915.905789
...,...,...,...,...,...
15,2025-06-01,None,1,NaN,NaN
16,2025-07-01,Beauty & Health,8,4928.88,616.110000
17,2025-07-01,"Bed, Bath & Table",1,231.87,231.870000
18,2025-07-01,Car,16,25282.41,1580.150625


In [22]:
df_category_month = thread.df()
df_category_month.head()

,month,product_category_name,orders_count,revenue_total,aov
0,2025-06-01,Beauty & Health,8,6088.22,761.027500
1,2025-06-01,"Bed, Bath & Table",8,9505.27,1188.158750
2,2025-06-01,Car,8,11747.85,1468.481250
3,2025-06-01,Computers,12,24325.25,2027.104167
4,2025-06-01,Cool Stuff,19,36402.21,1915.905789


In [23]:
# Aggregate by seller overall
thread.ask(
    """
    Aggregate performance by seller_id using orders_features
    """
)


======== <THINKING> ========

**Aggregating seller performance metrics**

The user wants to create an aggregation table by seller_id using the orders_features temp table. I'll select reasonable metrics like orders_count, revenue_total, avg_order_value, items_count_sum, and others while ensuring I keep explanations concise. I'll use SQL to create a temp table for seller performance and return a sample DataFrame. I need to make sure I recall the relevant columns from the orders_features table and compute averages correctly in DuckDB, especially converting is_late to a double for accurate calculations.**Creating seller performance table**

I'm working with integer values of 0 and 1, and I know DuckDB's AVG will automatically cast these to double, which is good! Just to be cautious, I have to pay attention to NaN values for discount_pct, but it’s numeric. For calculating avg_discount_pct, I can use AVG(discount_pct) while ignoring nulls. I need to create the temp table seller_performance b

,seller_id,seller_city,seller_state,orders_count,customers_count,revenue_total,gmv_total,avg_order_value,avg_order_gmv,avg_items_per_order,avg_delivery_days,avg_delivery_delay_days,late_rate,review_coverage_rate,positive_review_rate,negative_review_rate,high_value_order_share,avg_discount_pct,avg_freight_share_pct
0,sell_0029,heerlen,LI,16,16,24846.45,25516.54,1552.903125,1594.783750,3.250000,9.500000,-0.400000,0.312500,0.187500,0.062500,0.125000,0.875000,4.119946,7.600286
1,sell_0012,zoetermeer,ZH,14,14,23733.86,24532.22,1695.275714,1752.301429,2.857143,11.222222,0.444444,0.285714,0.214286,0.071429,0.142857,0.642857,9.187532,11.914043
2,sell_0018,rotterdam,ZH,15,15,22972.72,23401.34,1531.514667,1560.089333,2.600000,10.700000,0.300000,0.266667,0.133333,0.133333,0.000000,0.666667,3.236705,6.500067
3,sell_0050,emmmen,DR,11,11,21387.74,21786.29,1944.340000,1980.571818,2.818182,10.444444,-1.555556,0.454545,0.363636,0.090909,0.272727,0.727273,3.765769,9.245448
4,sell_0023,maastricht,LI,18,17,20249.07,20346.34,1124.948333,1130.352222,2.000000,10.125000,0.250000,0.500000,0.555556,0.222222,0.277778,0.611111,11.301616,10.860973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,sell_0021,roermond,LI,8,8,17045.13,17293.95,2130.641250,2161.743750,3.875000,8.714286,-1.571429,0.375000,0.500000,0.375000,0.125000,0.875000,3.706994,7.356593
16,sell_0036,nieuwegein,UT,16,15,16927.94,17171.68,1057.996250,1073.230000,2.250000,9.875000,0.750000,0.250000,0.250000,0.125000,0.125000,0.437500,7.605778,8.989460
17,sell_0014,arnhem,GE,9,9,16046.35,16354.46,1782.927778,1817.162222,2.555556,11.750000,1.125000,0.555556,0.444444,0.111111,0.333333,0.666667,3.997566,5.865549
18,sell_0013,hilversum,UT,9,9,15867.66,16248.61,1763.073333,1805.401111,3.000000,10.333333,-8.333333,0.000000,0.000000,0.000000,0.000000,0.666667,10.874328,14.052310


In [24]:
df_seller_kpis = thread.df()
df_seller_kpis.head()


,seller_id,seller_city,seller_state,orders_count,customers_count,revenue_total,gmv_total,avg_order_value,avg_order_gmv,avg_items_per_order,avg_delivery_days,avg_delivery_delay_days,late_rate,review_coverage_rate,positive_review_rate,negative_review_rate,high_value_order_share,avg_discount_pct,avg_freight_share_pct
0,sell_0029,heerlen,LI,16,16,24846.45,25516.54,1552.903125,1594.783750,3.250000,9.500000,-0.400000,0.312500,0.187500,0.062500,0.125000,0.875000,4.119946,7.600286
1,sell_0012,zoetermeer,ZH,14,14,23733.86,24532.22,1695.275714,1752.301429,2.857143,11.222222,0.444444,0.285714,0.214286,0.071429,0.142857,0.642857,9.187532,11.914043
2,sell_0018,rotterdam,ZH,15,15,22972.72,23401.34,1531.514667,1560.089333,2.600000,10.700000,0.300000,0.266667,0.133333,0.133333,0.000000,0.666667,3.236705,6.500067
3,sell_0050,emmmen,DR,11,11,21387.74,21786.29,1944.340000,1980.571818,2.818182,10.444444,-1.555556,0.454545,0.363636,0.090909,0.272727,0.727273,3.765769,9.245448
4,sell_0023,maastricht,LI,18,17,20249.07,20346.34,1124.948333,1130.352222,2.000000,10.125000,0.250000,0.500000,0.555556,0.222222,0.277778,0.611111,11.301616,10.860973


### Wrapping it up

- You just walked through the complete data preparation workflow in Databao: Understanding → Cleaning → Integration → Feature Engineering → Aggregation & Export.
- You used both per‑source and project‑wide contexts to guide the LLM and ensure consistent results.
- You used `.ask()` in the default eager mode to materialize results incrementally. You can also try to chain multiple asks and compute them at once in a new thread with `lazy=True`.
- Your final datasets are now analytics-ready and suitable for dashboards, exploratory analysis, or downstream machine-learning pipelines.
